[List of countries by life expectancy](https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy#United_Nations_(2023)) / [Список стран по ожидаемой продолжительности жизни](https://ru.wikipedia.org/wiki/Список_стран_по_ожидаемой_продолжительности_жизни)<br>
data source: [World Population Prospects - United Nations](https://population.un.org/wpp/Download/Standard/MostUsed/)

In [1]:
import pandas as pd
import math
import re
from collections import namedtuple

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [2]:
# If database process the first time or was updated, set the constant as True - intermediate calculations will be saved
# to dist to speed up further processing.
# If the constant is set as False, data will be taken from file with intermediate calculations but not from the original database.
IS_FIRST_ANALYSIS = True

DESTINATION_OUTPUT = 'file'  # to where table code should be placed: 'file', 'here', 'print_done'

pd.options.display.max_rows = 300

In [3]:
def output_table_code(st, file_name='', destination=DESTINATION_OUTPUT):

    if destination == 'file':
        with open('output/'+file_name, 'w', encoding="utf-8") as fh:
            fh.write(st)
        print('Data has written to file')
    elif destination == 'here':
        print(st)
    else:
        print('done')

In [4]:
# if data is loaded from the original database
if IS_FIRST_ANALYSIS:
    
    # load data from original XLSX-file and rename columns for convenience
    df = pd.read_excel('data/UN_data-2023.xlsx', sheet_name='Estimates', skiprows=16, na_values='...',
                   usecols=['Region, subregion, country or area *', 'Year', 'Total Population, as of 1 July (thousands)',
                            'Median Age, as of 1 July (years)', 'Life Expectancy at Birth, both sexes (years)',
                            'Male Life Expectancy at Birth (years)', 'Female Life Expectancy at Birth (years)']) \
       .rename(columns={'Region, subregion, country or area *': 'region',
                        'Year': 'year',
                        'Total Population, as of 1 July (thousands)': 'population',
                        'Median Age, as of 1 July (years)': 'median_age',
                        'Life Expectancy at Birth, both sexes (years)': 'le_total',
                        'Male Life Expectancy at Birth (years)': 'le_male',
                        'Female Life Expectancy at Birth (years)': 'le_female'})
    
    print(len(df))
    
    # drop records without data about life expectancy
    df.dropna(how='all', subset=['le_total', 'le_male', 'le_female'], inplace=True)
    
    print(len(df))

    # population was in thousands - remake it to number
    df['population'] = (df['population']*1000).astype('int64')

    # change order of columns
    df = df[['region', 'year', 'le_total', 'le_male', 'le_female', 'median_age', 'population']]
    
    # save intermediate data to file
    df.to_csv('data/UN_data-intermediate_calculations.csv', index=False)
    print("Intermediate data is written to file")

# if data is loaded from file with intermediate calculations
else:
    df = pd.read_csv('data/UN_data-intermediate_calculations.csv')
    print("Data is loaded from file with intermediate data")
    
df.head(3)

21983
21978
Intermediate data is written to file


,region,year,le_total,le_male,le_female,median_age,population
0,World,1950.0,46.394,44.465,48.436,22.159,2493092848
1,World,1951.0,47.126,45.300,49.040,22.123,2536927035
2,World,1952.0,48.218,46.525,49.972,22.074,2584086339


In [5]:
# set region names as index
df.set_index('region', inplace=True)

df.index.name = ''

In [6]:
# take only interesting values to form new kind of dataframe
df = pd.concat([
        df.loc[df.year==2014]['le_total'],
        df.loc[df.year==2019]['le_total'],
        df.loc[df.year==2020]['le_total'],
        df.loc[df.year==2021]['le_total'],
        df.loc[df.year==2022]['le_total'],
        df.loc[df.year==2023]['le_total'],
        df.loc[df.year==2023]['le_male'],
        df.loc[df.year==2023]['le_female'],
        df.loc[df.year==2023]['median_age'],
        df.loc[df.year==2023]['population']
], axis='columns')

df.columns = ['2014_t', '2019_t', '2020_t', '2021_t', '2022_t', 'total', 'male', 'female', 'median_age', 'population']

print(len(df))
df.head(3)

297


,2014_t,2019_t,2020_t,2021_t,2022_t,total,male,female,median_age,population
,,,,,,,,,,
World,71.404,72.609,71.917,70.865,72.640,73.169,70.547,75.886,30.364,8091734930
Sub-Saharan Africa,58.359,60.499,60.601,60.109,61.090,62.120,60.116,64.140,18.078,1212229420
Northern Africa and Western Asia,72.049,73.356,72.078,71.707,73.546,73.768,71.480,76.175,25.872,572506537


In [7]:
# drop some records and rename others
df = df.drop([
            'Europe, Northern America, Australia, and New Zealand',
            'More developed regions',
            'Less developed regions',
            'Least developed countries',
            'Less developed regions, excluding least developed countries',
            'Less developed regions, excluding China',
            'Land-locked Developing Countries (LLDC)',
            'LLDC: Africa',
            'LLDC: Asia',
            'LLDC: Europe',
            'LLDC: Latin America',
            'Small Island Developing States (SIDS)',
            'SIDS Caribbean',
            'SIDS Pacific',
            'SIDS Atlantic, Indian Ocean and South China Sea (AIS)',
            'High-and-upper-middle-income countries',
            'Low-and-Lower-middle-income countries',
            'Low-and-middle-income countries',
            'No income group available']) \
        .rename(index = {
            'United Republic of Tanzania': 'Tanzania',
            'Congo': 'Congo, Rep.',
            'Democratic Republic of the Congo': 'DR Congo',
            "Côte d'Ivoire": "Cote d'Ivoire",
            'China, Hong Kong SAR': 'Hong Kong, China',
            'China, Macao SAR': 'Macao, China',
            'China, Taiwan Province of China': 'Taiwan',
            "Dem. People's Republic of Korea": "North Korea",
            'Republic of Korea': 'South Korea',
            'Iran (Islamic Republic of)': 'Iran',
            'Brunei Darussalam': 'Brunei',
            "Lao People's Democratic Republic": 'Laos',
            'Viet Nam': 'Vietnam',
            'State of Palestine': 'Palestine',
            'Syrian Arab Republic': 'Syria',
            'Türkiye': 'Turkey',
            'Republic of Moldova': 'Moldova',
            'Russian Federation': 'Russia',
            'Kosovo (under UNSC res. 1244)': 'Kosovo',
            'Saint Martin (French part)': 'Saint Martin',
            'Sint Maarten (Dutch part)': 'Sint Maarten',
            'Bolivia (Plurinational State of)': 'Bolivia',
            'Venezuela (Bolivarian Republic of)': 'Venezuela',
            'Micronesia (Fed. States of)': 'F.S. Micronesia',
            'Micronesia': 'Micronesia (region)',
            'United States of America': 'USA',
            'United Arab Emirates': 'UAE',
            'Central African Republic': 'CAR',
            'Falkland Islands (Malvinas)': 'Falkland Islands',
            'Bonaire, Sint Eustatius and Saba': 'Caribbean Netherlands',
            'United States Virgin Islands': 'US Virgin Islands'}) \
        .drop_duplicates() \
        .sort_values(by=['total', 'male', 'female'], ascending=False)

print(len(df))
df.head(3)

276


,2014_t,2019_t,2020_t,2021_t,2022_t,total,male,female,median_age,population
,,,,,,,,,,
Monaco,85.093,86.151,86.089,85.113,85.746,86.372,84.448,88.504,54.356,38956
San Marino,84.394,85.257,82.650,83.539,85.708,85.706,84.206,87.100,47.393,33733
"Hong Kong, China",83.927,85.263,84.707,85.082,83.485,85.511,82.844,88.129,46.155,7442734


In [8]:
# calculate change between periods and also sex gap
df.insert(loc=1, column='2014→2019', value=(df['2019_t']-df['2014_t']).round(3))
df.insert(loc=3, column='2019→2020', value=(df['2020_t']-df['2019_t']).round(3))
df.insert(loc=5, column='2020→2021', value=(df['2021_t']-df['2020_t']).round(3))
df.insert(loc=7, column='2021→2022', value=(df['2022_t']-df['2021_t']).round(3))
df.insert(loc=9, column='2022→2023', value=(df['total']-df['2022_t']).round(3))
df.insert(loc=11, column='2019→2023', value=(df['total']-df['2019_t']).round(3))
df.insert(loc=12, column='2014→2023', value=(df['total']-df['2014_t']).round(3))
df.insert(loc=15, column='sex_gap', value=(df['female']-df['male']).round(3))
df.head()

,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2021_t,2021→2022,2022_t,2022→2023,total,2019→2023,2014→2023,male,female,sex_gap,median_age,population
,,,,,,,,,,,,,,,,,,
Monaco,85.093,1.058,86.151,-0.062,86.089,-0.976,85.113,0.633,85.746,0.626,86.372,0.221,1.279,84.448,88.504,4.056,54.356,38956
San Marino,84.394,0.863,85.257,-2.607,82.650,0.889,83.539,2.169,85.708,-0.002,85.706,0.449,1.312,84.206,87.100,2.894,47.393,33733
"Hong Kong, China",83.927,1.336,85.263,-0.556,84.707,0.375,85.082,-1.597,83.485,2.026,85.511,0.248,1.584,82.844,88.129,5.285,46.155,7442734
Japan,83.643,0.775,84.418,0.252,84.670,-0.120,84.550,-0.496,84.054,0.658,84.712,0.294,1.069,81.688,87.741,6.053,48.958,124370947
South Korea,82.364,1.321,83.685,-0.010,83.675,0.177,83.852,-1.125,82.727,1.602,84.329,0.644,1.965,81.194,87.158,5.964,44.486,51748739


<br>
<br>

In [9]:
# determine list of regions (to exclude them from dataFrame when it will be necessary)
ls_regions = [
    'Sub-Saharan Africa',
    'Northern Africa and Western Asia',
    'Central and Southern Asia',
    'Eastern and South-Eastern Asia',
    'Latin America and the Caribbean',
    'Oceania (excluding Australia and New Zealand)',
    'Australia/New Zealand',
    'Asia',
    'Central Asia',
    'Eastern Asia',
    'Southern Asia',
    'South-Eastern Asia',
    'Western Asia',
    'Africa',
    'Eastern Africa',
    'Middle Africa',
    'Northern Africa',
    'Southern Africa',
    'Western Africa',
    'Europe',
    'Eastern Europe',
    'Northern Europe',
    'Southern Europe',
    'Western Europe',
    'Central America',
    'South America',
    'Northern America',
    'Europe and Northern America',
    'Caribbean',
    'Oceania',
    'Polynesia',
    'Melanesia',
    'Micronesia (region)',
    'High-income countries',
    'Upper-middle-income countries',
    'Middle-income countries',
    'Lower-middle-income countries',
    'Low-income countries'
]

In [25]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
mal.min_and_max_values(df.drop(ls_regions).loc[:, '2014_t':'sex_gap'],
                       row_center=['Spain', 'France', 'World'], nmb=5, max_lng=10, shorten=True)

Number of records: 238


,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2021_t,2021→2022,2022_t,2022→2023,total,2019→2023,2014→2023,male,female,sex_gap
max,85.09 -Monaco,12.5 -S. Sudan,86.15 -Monaco,19.07 -CAR,86.09 -Monaco,3.66 -Armenia,85.11 -Monaco,6.01 -Bolivia,85.75 -Monaco,38.59 -CAR,86.37 -Monaco,25.88 -CAR,17.14 -CAR,84.45 -Monaco,88.5 -Monaco,13.3 -Ukraine
max_2,84.48 -Andorra,6.36 -Eswatini,85.26 -Hong Kong,1.97 -Seychelles,84.71 -Hong Kong,2.91 -Andorra,85.08 -Hong Kong,5.57 -Guyana,85.71 -San Marino,4.88 -Somalia,85.71 -San Marino,4.27 -Eswatini,11.99 -S. Sudan,84.21 -San Marino,88.13 -Hong Kong,11.81 -Palestine
max_3,84.39 -San Marino,5.48 -Syria,85.26 -San Marino,1.59 -Bahamas,84.67 -Japan,1.95 -Algeria,84.55 -Japan,5.45 -Botswana,84.28 -Saint Bar…,3.19 -Namibia,85.51 -Hong Kong,3.84 -Namibia,10.63 -Eswatini,82.84 -Hong Kong,87.74 -Japan,11.78 -Russia
max_4,83.93 -Hong Kong,5.31 -Sierra Le…,84.42 -Japan,1.32 -Macao,84.61 -Macao,1.05 -Iraq,84.19 -Saint Bar…,5.24 -Peru,84.05 -Japan,2.42 -UAE,84.71 -Japan,3.44 -Zambia,8.07 -Namibia,82.1 -Andorra,87.16 -S. Korea,10.84 -US Virgin…
max_5,83.64 -Japan,5.0 -Botswana,84.1 -Andorra,"1.24 -Congo, Re…",84.07 -Saint Bar…,0.99 -Tajikistan,83.85 -S. Korea,5.2 -Afghanist…,84.02 -Andorra,2.41 -Brunei,84.33 -S. Korea,3.14 -Bahamas,7.5 -Sierra Le…,82.1 -Australia,87.1 -San Marino,9.54 -Georgia
Spain,– 82.83 –,– 0.65 –,– 83.48 –,– -1.24 –,– 82.24 –,– 0.71 –,– 82.95 –,– -0.58 –,– 82.37 –,– 1.3 –,– 83.67 –,– 0.19 –,– 0.84 –,– 80.96 –,– 86.31 –,– 5.35 –
France,– 82.44 –,– 0.29 –,– 82.73 –,– -0.53 –,– 82.2 –,– 0.12 –,– 82.32 –,– 0.15 –,– 82.47 –,– 0.85 –,– 83.33 –,– 0.59 –,– 0.89 –,– 80.43 –,– 86.09 –,– 5.66 –
World,– 71.4 –,– 1.21 –,– 72.61 –,– -0.69 –,– 71.92 –,– -1.05 –,– 70.86 –,– 1.77 –,– 72.64 –,– 0.53 –,– 73.17 –,– 0.56 –,– 1.76 –,– 70.55 –,– 75.89 –,– 5.34 –
min_5,51.94 -Nigeria,-0.87 -Samoa,57.25 -Somalia,-4.08 -Mexico,57.09 -Somalia,-3.43 -Guyana,55.7 -Somalia,-1.72 -Macao,56.81 -Lesotho,-0.49 -Dominican…,57.62 -S. Sudan,-0.58 -Turkey,-0.88 -Jamaica,55.26 -CAR,60.63 -S. Sudan,1.87 -Niger
min_4,51.14 -Chad,-1.18 -Mayotte,55.25 -Lesotho,-4.68 -Andorra,55.13 -Lesotho,-4.21 -Cuba,54.21 -Lesotho,-1.77 -Somalia,54.53 -Chad,-0.67 -Syria,57.41 -CAR,-0.64 -Holy See,-0.97 -Jersey,54.64 -S. Sudan,60.01 -Lesotho,1.76 -Qatar


<br />
<br />

In [11]:
# create code for Wikipedia table
def create_table_code(df, lang='en', with_population=True):

    def if_value(x, prec=2):
        # decoration of regular values
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=2):  # 'change_value'
        # decoration of colored values, that shows change of regular values
        return '"| —' if math.isnan(x) else \
               f' color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f' color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f' color:darkgray;"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=2):
        # decoration of colored values of bold font
        return '"| —' if math.isnan(x) else \
               f' color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f' color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f' color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    def population_prettify(pop):
        # decoration of population number, represent values in thousands
        pop = int(round(pop/1000))
        if pop == 0:
            pop = '<500'
        return pop
    
    # depending on language of created table, some elements of the table will have various elements
    if lang=='ru':
        file_header='un_header_2023_ru_with_population.txt' if with_population else 'un_header_2023_ru.txt'
        ptn_1 = 'флагификация'
        ptn_2 = 'флаг'
        prettify_name = {
            'World': 'Мир',
            'Europe': '[[Европа]]',
            'Asia': '[[Азия]]',
            'Northern America': '[[Северная Америка (регион)|Северная Америка]]',
            'Central America': '[[Центральная Америка]]',
            'South America': '[[Южная Америка]]',
            'Caribbean': '[[Карибские острова]]',
            'Latin America and the Caribbean': '[[Латинская Америка]] и [[Антильские острова|Карибы]]',
            'Africa': '[[Африка]]',
            'Taiwan': '[[Тайвань]]',
            'Kosovo': '[[Республика Косово|Косово]]',
            'Falkland Islands': '[[Фолклендские острова]]',
            'Western Sahara': '[[Западная Сахара]]' 
        }
    else:
        file_header='un_header_2023_en_with_population.txt' if with_population else 'un_header_2023_en.txt'
        ptn_1 = 'flaglist'
        ptn_2 = 'flagicon'
        prettify_name = {
            'World': 'World',
            'Europe': '[[Europe]]',
            'Asia': '[[Asia]]',
            'Northern America': '[[Northern America]]',
            'Central America': '[[Central America]]',
            'South America': '[[South America]]',
            'Caribbean': '[[Caribbean]]',
            'Latin America and the Caribbean': '[[Latin America and the Caribbean|Latin America & Caribbean]]',
            'Africa': '[[Africa]]',
            'Taiwan': '[[Taiwan]]',
            'Kosovo': '[[Kosovo]]',
            'Falkland Islands': '[[Falkland Islands]]',
            'Western Sahara': '[[Western Sahara]]'
        }
    
    # Load code for header of the table from file. Depending on condions (see above), code is loaded from various files.
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        st_header = fh.read().strip()
    
    # Create table code record by record.
    # There are 3 types of record that are generated a little different.
    # Some elements can be different depending on language of the table.
    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        
        # records in bold
        if ser.name in ['World',
                        'Europe',
                        'Asia',
                        'Northern America',
                        'Central America',
                        'Caribbean',
                        'South America',
                        'Latin America and the Caribbean',
                        'Oceania',
                        'Africa']:
            st += '\n' + '|-class=static-row-header\n' + \
                  f'|align=center| \'\'\'{prettify_name.get(ser.name, ser.name)}\'\'\' ' + \
                  f'||style="text-align:center;background:#e0ffd8;"| \'\'\'{if_value(ser.total, 2)}\'\'\' ' + \
                  f'||style="text-align:center;background:#eaf3ff;"| \'\'\'{if_value(ser.male, 2)}\'\'\' ' + \
                  f'||style="text-align:center;background:#fee7f6;"| \'\'\'{if_value(ser.female, 2)}\'\'\' ' + \
                  f'||style="text-align:center;background:#fff8dc;"| \'\'\'{if_value(ser.sex_gap, 2)}\'\'\' ' + \
                  (f'||style="text-align:center;border-left-width:2px;padding-left:1em;"| \'\'\'{if_value(ser["2014_t"], 2)}\'\'\' ' + \
                  f'||style="{chval_bold(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2019_t"], 2)}\'\'\' ' if lang=='ru' else \
                  f'||style="text-align:center;border-left-width:2px;padding-left:1em;"| \'\'\'{if_value(ser["2019_t"], 2)}\'\'\' ') + \
                  f'||style="{chval_bold(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2020_t"], 2)}\'\'\' ' + \
                  f'||style="{chval_bold(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2021_t"], 2)}\'\'\' ' + \
                  f'||style="{chval_bold(ser["2021→2022"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2022_t"], 2)}\'\'\' ' + \
                  f'||style="{chval_bold(ser["2022→2023"], 2)} ' + \
                  f'||style="text-align:center;background:#e0ffd8;"| \'\'\'{if_value(ser.total, 2)}\'\'\' ' + \
                  (f'||style="background:#fffae0;border-left-width:2px;padding-right:1em;{chval_bold(ser["2019→2023"], 2)} ' + \
                   f'||style="background:#fffae0;padding-right:1em;{chval_bold(ser["2014→2023"], 2)} ' if lang=='ru' else \
                   f'||style="background:#fffae0;border-left-width:2px;padding-right:2em;{chval_bold(ser["2019→2023"], 2)} ') + \
                  (f'||style="border-left-width:2px;| \'\'\'{population_prettify(ser.population)}\'\'\' ' if with_population else '') + \
                  f'||'
        # records without flags - are used when territories are objects of territorial disputes, or some collective objects (e.g. 'Caribbean small states')
        elif ser.name in ['Kosovo',
                          'Taiwan',
                          'Falkland Islands',
                          'Western Sahara']:
            st += '\n' + '|-\n' + \
                  f'|style="text-align:left;padding-left: 35px;"|{prettify_name.get(ser.name, ser.name)} ' + \
                  f'||style="text-align:center;background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||style="text-align:center;background:#eaf3ff;"| {if_value(ser.male, 2)} ' + \
                  f'||style="text-align:center;background:#fee7f6;"| {if_value(ser.female, 2)} ' + \
                  f'||style="text-align:center;background:#fff8dc;"| {if_value(ser.sex_gap, 2)} ' + \
                  (f'||style="text-align:center;border-left-width:2px;padding-left:1em;"| {if_value(ser["2014_t"], 2)} ' + \
                  f'||style="{chval(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2019_t"], 2)} ' if lang=='ru' else \
                  f'||style="text-align:center;border-left-width:2px;padding-left:1em;"| {if_value(ser["2019_t"], 2)} ') + \
                  f'||style="{chval(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2020_t"], 2)} ' + \
                  f'||style="{chval(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2021_t"], 2)} ' + \
                  f'||style="{chval(ser["2021→2022"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2022_t"], 2)} ' + \
                  f'||style="{chval(ser["2022→2023"], 2)} ' + \
                  f'||style="text-align:center;background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  (f'||style="background:#fffae0;border-left-width:2px;padding-right:1em;{chval(ser["2019→2023"], 2)} ' + \
                   f'||style="background:#fffae0;padding-right:1em;{chval(ser["2014→2023"], 2)} ' if lang=='ru' else \
                   f'||style="background:#fffae0;border-left-width:2px;padding-right:2em;{chval(ser["2019→2023"], 2)} ') + \
                  (f'||style="border-left-width:2px;| {population_prettify(ser.population)} ' if with_population else '') + \
                  f'||'
        # regular records
        else:
            st += '\n' + '|-\n' + \
                  f'|align=left|{{{{{ptn_1}|{ser.name}}}}} ' + \
                  f'||style="text-align:center;background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||style="text-align:center;background:#eaf3ff;"| {if_value(ser.male, 2)} ' + \
                  f'||style="text-align:center;background:#fee7f6;"| {if_value(ser.female, 2)} ' + \
                  f'||style="text-align:center;background:#fff8dc;"| {if_value(ser.sex_gap, 2)} ' + \
                  (f'||style="text-align:center;border-left-width:2px;padding-left:1em;"| {if_value(ser["2014_t"], 2)} ' + \
                   f'||style="{chval(ser["2014→2019"], 2)} ' + \
                   f'||style="text-align:center;"| {if_value(ser["2019_t"], 2)} ' if lang=='ru' else \
                   f'||style="text-align:center;border-left-width:2px;padding-left:1em;"| {if_value(ser["2019_t"], 2)} ') + \
                  f'||style="{chval(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2020_t"], 2)} ' + \
                  f'||style="{chval(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2021_t"], 2)} ' + \
                  f'||style="{chval(ser["2021→2022"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2022_t"], 2)} ' + \
                  f'||style="{chval(ser["2022→2023"], 2)} ' + \
                  f'||style="text-align:center;background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  (f'||style="background:#fffae0;border-left-width:2px;padding-right:1em;{chval(ser["2019→2023"], 2)} ' + \
                   f'||style="background:#fffae0;padding-right:1em;{chval(ser["2014→2023"], 2)} ' if lang=='ru' else \
                   f'||style="background:#fffae0;border-left-width:2px;padding-right:2em;{chval(ser["2019→2023"], 2)} ') + \
                  (f'||style="border-left-width:2px;| {population_prettify(ser.population)} ' if with_population else '') + \
                  f'||align=center|{{{{{ptn_2}|{ser.name}}}}}'
    st += '\n|}'
    
    # fix microbugs that can happened due to nuances of calculations and rounding
    st = st.replace('color:darkgray;"| 0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:crimson;"| −0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:darkgreen;"| 0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:darkgray;"| \'\'\'0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')  \
           .replace('color:crimson;"| \'\'\'−0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')  \
           .replace('color:darkgreen;"| \'\'\'0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')
      
    # since in Russian standard decimal numbers use a comma, made in numbers replacement of dot to comma
    # WORKING WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits
    
    # resolve issue with specific flag of New Caledonia
    if lang == 'ru':
        st = st.replace('{{флагификация|New Caledonia}}', '{{флагификация|New Caledonia|совмещённый_укороченный}}')
        st = st.replace('{{флаг|New Caledonia}}', '{{флаг|New Caledonia|совмещённый_укороченный}}')
    else:
        st = st.replace('{{flaglist|New Caledonia}}', '{{flaglist|New Caledonia|merged}}')
        st = st.replace('{{flagicon|New Caledonia}}', '{{flagicon|New Caledonia|merged}}')
        
    # set gray color of text for missing values that in the table are represented with '—'
    st = st.replace(';"| —', ';color:silver;"| —')
    
    # join code of header of the table with code for records
    st = st_header + st
    
    return st

In [12]:
print("Number of all countries and territories in the list:", len(df[(~df.index.isin(ls_regions))]))

Number of all countries and territories in the list: 238


<br />
<br />

Code for main table (with all countries where population >=50,000. Column with population is not shown here.

In [13]:
df_all_countries = df[(~df.index.isin(ls_regions)) & (df.population>=50_000)]
print(len(df_all_countries))

table_code = create_table_code(df_all_countries, with_population=False)
output_table_code(table_code, 'Table code UN -all_countries -en.txt')

table_code = create_table_code(df_all_countries, lang='ru', with_population=False)
output_table_code(table_code, 'Table code UN -all_countries -ru.txt')

211
Data has written to file
Data has written to file


<br />
<br />

Code for table with European countries. Column with population is shown.

In [14]:
ls_Europe = ["Albania", "Andorra", "Armenia", "Azerbaijan", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czechia",
             "Denmark", "Estonia", "Faroe Islands", "Finland", "France", "Germany", "Gibraltar", "Georgia", "Greece", "Guernsey", "Holy See", "Hungary", "Iceland",
             "Ireland", "Isle of Man", "Italy", "Jersey", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Malta",
             "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", "Romania", "Russia",
             "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Ukraine", "United Kingdom",
             "Europe", "World"]

df_Europe = df[(df.index.isin(ls_Europe))]

print(len(df_Europe))
assert len(ls_Europe) == len(df_Europe)

table_code = create_table_code(df_Europe, with_population=True)
output_table_code(table_code, 'Table code UN -Europe -en.txt')

table_code = create_table_code(df_Europe, lang='ru', with_population=True)
output_table_code(table_code, 'Table code UN -Europe -ru.txt')

56
Data has written to file
Data has written to file


<br />
<br />

In [15]:
ls_Asia = ["Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Cambodia", "China", "Cyprus", "Georgia",
           "Hong Kong, China", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", "Kazakhstan", "Kuwait", "Kyrgyzstan",
           "Laos", "Lebanon", "Macao, China", "Malaysia", "Maldives", "Mongolia", "Myanmar", "Nepal", "North Korea", "Oman", "Pakistan",
           "Palestine", "Philippines", "Qatar", "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Taiwan", "Tajikistan",
           "Thailand", "Timor-Leste", "Turkey", "Turkmenistan", "UAE", "Uzbekistan", "Vietnam", "Yemen",
           "Asia", "World"]  # "United Arab Emirates"

df_Asia = df[(df.index.isin(ls_Asia))]

print(len(df_Asia))
assert len(ls_Asia) == len(df_Asia)

table_code = create_table_code(df_Asia, with_population=True)
output_table_code(table_code, 'Table code UN -Asia -en.txt')

table_code = create_table_code(df_Asia, lang='ru', with_population=True)
output_table_code(table_code, 'Table code UN -Asia -ru.txt')

53
Data has written to file
Data has written to file


<br />
<br />

In [16]:
ls_North_America = ["Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados", "Belize", "Bermuda", "British Virgin Islands",
                    "Canada", "Caribbean Netherlands", "Cayman Islands", "Costa Rica", "Cuba", "Curaçao", "Dominica", "Dominican Republic",
                    "El Salvador", "Greenland", "Grenada", "Guadeloupe", "Guatemala", "Haiti",
                    "Honduras", "Jamaica", "Martinique", "Mexico", "Montserrat", "Nicaragua", "Panama", "Puerto Rico", "Saint Barthélemy",
                    "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin", "Saint Pierre and Miquelon", "Saint Vincent and the Grenadines",
                    "Sint Maarten", "Trinidad and Tobago", "Turks and Caicos Islands", "US Virgin Islands", "USA",
                    "World"]  # "Northern America", "Central America", "Caribbean", 

df_North_America = df[(df.index.isin(ls_North_America))]

print(len(df_North_America))
assert len(ls_North_America) == len(df_North_America)

table_code = create_table_code(df_North_America, with_population=True)
output_table_code(table_code, 'Table code UN -America North -en.txt')

table_code = create_table_code(df_North_America, lang='ru', with_population=True)
output_table_code(table_code, 'Table code UN -America North -ru.txt')

42
Data has written to file
Data has written to file


<br />
<br />

In [17]:
ls_South_America = ["Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Falkland Islands",
                    "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela",
                    "South America", "World"]

df_South_America = df[(df.index.isin(ls_South_America))]

print(len(df_South_America))
assert len(ls_South_America) == len(df_South_America)

table_code = create_table_code(df_South_America, with_population=True)
output_table_code(table_code, 'Table code UN -America South -en.txt')

table_code = create_table_code(df_South_America, lang='ru', with_population=True)
output_table_code(table_code, 'Table code UN -America South -ru.txt')

16
Data has written to file
Data has written to file


<br />
<br />

In [18]:
ls_Americas = ls_North_America + ls_South_America
ls_Americas.remove("South America")
ls_Americas.remove("World")

df_Americas = df[(df.index.isin(ls_Americas))]

print(len(df_Americas))
assert len(ls_Americas) == len(df_Americas)

table_code = create_table_code(df_Americas, with_population=True)
output_table_code(table_code, 'Table code UN -Americas -en.txt')

# table_code = create_table_code(df_Americas, lang='ru', with_population=True)
# output_table_code(table_code, 'Table code UN -Americas -ru.txt')

56
Data has written to file


<br />
<br />

In [19]:
ls_Oceania = ["American Samoa", "Australia", "Cook Islands", "Fiji", "French Polynesia", "F.S. Micronesia", "Guam", "Kiribati", "Marshall Islands",
              "Nauru", "New Caledonia", "New Zealand", "Niue", "Northern Mariana Islands", "Palau", "Papua New Guinea",
              "Samoa", "Solomon Islands", "Tokelau", "Tonga", "Tuvalu", "Vanuatu", "Wallis and Futuna Islands",
              "Oceania", "World"]

df_Oceania = df[(df.index.isin(ls_Oceania))]

print(len(df_Oceania))
assert len(ls_Oceania) == len(df_Oceania)

table_code = create_table_code(df_Oceania, with_population=True)
output_table_code(table_code, 'Table code UN -Oceania -en.txt')

table_code = create_table_code(df_Oceania, lang='ru', with_population=True)
output_table_code(table_code, 'Table code UN -Oceania -ru.txt')

25
Data has written to file
Data has written to file


<br />
<br />

In [20]:
ls_Africa = ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde", "Cameroon", "CAR", "Chad", "Comoros", "Congo, Rep.",
             "Cote d'Ivoire", "Djibouti", "DR Congo", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana",
             "Guinea", "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Mayotte",
             "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Réunion", "Rwanda", "Saint Helena", "Sao Tome and Principe", "Senegal", "Seychelles",
             "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo", "Tunisia", "Uganda", "Western Sahara","Zambia", "Zimbabwe",
             "Africa", "World"]

df_Africa = df[(df.index.isin(ls_Africa))]

print(len(df_Africa))
assert len(ls_Africa) == len(df_Africa)

table_code = create_table_code(df_Africa, with_population=True)
output_table_code(table_code, 'Table code UN -Africa -en.txt')

table_code = create_table_code(df_Africa, lang='ru', with_population=True)
output_table_code(table_code, 'Table code UN -Africa -ru.txt')

60
Data has written to file
Data has written to file


<br />
<br />
<br />

---

Creation of code for [Europe map](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy#United_Nations_(2023))

In [21]:
MapLocations = namedtuple('MapLocations', ['country', 'x', 'y', 'ru_name'],
                                        defaults=('-', 0, 0, '-'))

In [22]:
dd_coloring = {
    68: 'c00000',
    69: 'e00000',    
    70: 'ff0000',
    71: 'ff2000',
    72: 'ff4000',
    73: 'ff6000',
    74: 'ff8000',
    75: 'ffa000',
    76: 'ffc000',
    77: 'ffe000',
    78: 'ffff00',
    79: 'e0ff00',
    80: 'c0ff00',
    81: 'a0ff00',
    82: '80ff00',
    83: '60ff00',
    84: '40ff00',
    85: '20ff00',
    86: '00ff00'}

ls_locations = [
    MapLocations(country='Albania', x=0.615, y=0.665, ru_name='Албания'),
    MapLocations(country='Armenia', x=0.92, y=0.58, ru_name='Армения'),
    MapLocations(country='Austria', x=0.545, y=0.555, ru_name='Австрия'),
    MapLocations(country='Azerbaijan', x=0.96, y=0.56, ru_name='Азербайджан'),
    MapLocations(country='Belarus', x=0.67, y=0.45, ru_name='Белоруссия'),
    MapLocations(country='Belgium', x=0.43, y=0.49, ru_name='Бельгия'),
    MapLocations(country='Bosnia and Herzegovina', x=0.57, y=0.61, ru_name='Босния и Герцеговина'),
    MapLocations(country='Bulgaria', x=0.66, y=0.624, ru_name='Болгария'),
    MapLocations(country='Croatia', x=0.56, y=0.594, ru_name='Хорватия'),
    MapLocations(country='Cyprus', x=0.79, y=0.73, ru_name='Республика Кипр'),
    MapLocations(country='Czechia', x=0.545, y=0.518, ru_name='Чехия'),
    MapLocations(country='Denmark', x=0.48, y=0.42, ru_name='Дания'),
    MapLocations(country='Estonia', x=0.625, y=0.37, ru_name='Эстония'),
    MapLocations(country='Finland', x=0.63, y=0.30, ru_name='Финляндия'),
    MapLocations(country='France', x=0.40, y=0.55, ru_name='Франция'),
    MapLocations(country='Georgia', x=0.90, y=0.555, ru_name='Грузия'),
    MapLocations(country='Germany', x=0.495, y=0.496, ru_name='Германия'),
    MapLocations(country='Greece', x=0.64, y=0.68, ru_name='Греция'),
    MapLocations(country='Hungary', x=0.59, y=0.56, ru_name='Венгрия'),
    MapLocations(country='Iceland', x=0.34, y=0.23, ru_name='Исландия'),
    MapLocations(country='Ireland', x=0.34, y=0.43, ru_name='Ирландия'),
    MapLocations(country='Italy', x=0.53, y=0.65, ru_name='Италия'),
    MapLocations(country='Kazakhstan', x=0.97, y=0.39, ru_name='Казахстан'),
    MapLocations(country='Latvia', x=0.625, y=0.4, ru_name='Латвия'),
    MapLocations(country='Lithuania', x=0.625, y=0.43, ru_name='Литва'),
    MapLocations(country='Luxembourg', x=0.45, y=0.51, ru_name='Люксембург'),
    MapLocations(country='Malta', x=0.56, y=0.754, ru_name='Мальта'),
    MapLocations(country='Moldova', x=0.70, y=0.56, ru_name='Молдова'),
    MapLocations(country='Montenegro', x=0.59, y=0.63, ru_name='Черногория'),
    MapLocations(country='Netherlands', x=0.46, y=0.47, ru_name='Нидерланды'),
    MapLocations(country='North Macedonia', x=0.634, y=0.65, ru_name='Северная Македония'),
    MapLocations(country='Norway', x=0.49, y=0.34, ru_name='Норвегия'),
    MapLocations(country='Poland', x=0.58, y=0.48, ru_name='Польша'),
    MapLocations(country='Portugal', x=0.24, y=0.62, ru_name='Португалия'),
    MapLocations(country='Romania', x=0.655, y=0.575, ru_name='Румыния'),
    MapLocations(country='Russia', x=0.76, y=0.38, ru_name='Россия'),
    MapLocations(country='Serbia', x=0.615, y=0.605, ru_name='Сербия'),
    MapLocations(country='Slovakia', x=0.58, y=0.54, ru_name='Словакия'),
    MapLocations(country='Slovenia', x=0.535, y=0.58, ru_name='Словения'),
    MapLocations(country='Spain', x=0.3, y=0.65, ru_name='Испания'),
    MapLocations(country='Sweden', x=0.54, y=0.37, ru_name='Швеция'),
    MapLocations(country='Switzerland', x=0.474, y=0.56, ru_name='Швейцария'),
    MapLocations(country='Turkey', x=0.82, y=0.64, ru_name='Турция'),
    MapLocations(country='Ukraine', x=0.69, y=0.505, ru_name='Украина'),
    MapLocations(country='United Kingdom', x=0.38, y=0.46, ru_name='Великобритания')
]

# small coutries that are not included: 'Channel Islands', 'Faroe Islands', 'Gibraltar', 'Isle of Man'
# MapLocations(country='Liechtenstein', x=0.49, y=0.58, ru_name='Лихтенштейн'),

In [23]:
selected_col = 'total'
lang = 'ru'

for country, x, y, ru_name in ls_locations:
    # if country == 'Turkey':
    #     country = 'Türkiye'
        
    le = df.loc[country, selected_col].round(1)
    background = dd_coloring[int(le)]
    if country == 'Georgia':
        country = 'Georgia (country)'
    print(f'{{{{Image label small|x={x}|y={y}|scale=700|text=[[{ru_name if lang=="ru" else country}|<span style="background:#{background};color:black">{le}</span>]]}}}}')

{{Image label small|x=0.615|y=0.665|scale=700|text=[[Албания|<span style="background:#e0ff00;color:black">79.6</span>]]}}
{{Image label small|x=0.92|y=0.58|scale=700|text=[[Армения|<span style="background:#ffa000;color:black">75.7</span>]]}}
{{Image label small|x=0.545|y=0.555|scale=700|text=[[Австрия|<span style="background:#80ff00;color:black">82.0</span>]]}}
{{Image label small|x=0.96|y=0.56|scale=700|text=[[Азербайджан|<span style="background:#ff8000;color:black">74.4</span>]]}}
{{Image label small|x=0.67|y=0.45|scale=700|text=[[Белоруссия|<span style="background:#ff8000;color:black">74.4</span>]]}}
{{Image label small|x=0.43|y=0.49|scale=700|text=[[Бельгия|<span style="background:#80ff00;color:black">82.1</span>]]}}
{{Image label small|x=0.57|y=0.61|scale=700|text=[[Босния и Герцеговина|<span style="background:#ffe000;color:black">77.8</span>]]}}
{{Image label small|x=0.66|y=0.624|scale=700|text=[[Болгария|<span style="background:#ffa000;color:black">75.6</span>]]}}
{{Image label 

<br />
<br />

In [24]:
assert False

AssertionError: 

<br />
<br />

<span style="color: red;">Lists below are formed by ChartGPT and can don't include some territories - check is required<span>

In [ ]:
ls_EU = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland",
         "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden",
         "", "World"]

<br />
<br />

In [ ]:
ls_Middle_East = ["Bahrain", "Egypt", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman",
                  "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "UAE", "Yemen",
                  "", "World"]

<br />
<br />

In [ ]:
ls_Caribbean = ["Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados", "British Virgin Islands", "Caribbean Netherlands",
                "Cayman Islands", "Cuba", "Curaçao", "Dominica", "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica",
                "Martinique", "Montserrat", "Puerto Rico", "Saint Barthélemy", "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin",
                "Saint Pierre and Miquelon", "Saint Vincent and the Grenadines", "Sint Maarten", "Trinidad and Tobago",
                "Turks and Caicos Islands", "US Virgin Islands",
                "", "World"]

<br />
<br />

In [ ]:
ls_Central_America = ["Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Nicaragua", "Panama",
                      "", "World"]

<br />
<br />

In [ ]:
ls_Latin_America = [
    # Central America
    "Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Nicaragua", "Panama",

    # Mexico
    "Mexico",

    # Caribbean
    "Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados", "British Virgin Islands", "Cayman Islands", "Cuba", "Curaçao",
    "Dominica", "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica", "Martinique", "Montserrat", "Puerto Rico",
    "Saint Barthélemy", "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin", "Saint Pierre and Miquelon",
    "Saint Vincent and the Grenadines", "Sint Maarten", "Trinidad and Tobago", "Turks and Caicos Islands", "US Virgin Islands",

    # South America (excluding Suriname, Guyana, and the Falkland Islands)
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Paraguay", "Peru", "Uruguay", "Venezuela",
    
     "", "World"
]